# Projet Intelligence Artificielle

## Besoin client 3 : Système d'alerte pour les tempêtes

### Préparation des données

In [96]:
# Importation des librairies nécessaires

import pandas as pd
import numpy as np
import string
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [64]:
# Importation des bases de données 
data_prof = pd.read_csv('./Données/Data_Arbre.csv')
data_clean = pd.read_csv("./Données/data_clean.csv", encoding='utf-8', sep=";", decimal=",")

In [65]:
# Affichage des premières lignes des données 
data_prof.head()


,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,fk_pied,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37.0,EN PLACE,Jeune,semi libre,gazon,Alignement,Non,15.0,5.0,0.0,QUERUB,VILLE,Feuillu,Non
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,10.0,0.0,PINNIGnig,VILLE,Conifère,Non
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116.0,REMPLACÉ,Adulte,semi libre,gazon,Alignement,Non,30.0,10.0,0.0,ACEPSE,VILLE,Feuillu,Non
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,2.0,0.0,ACEPLA,VILLE,Feuillu,Non
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,gazon,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non


In [66]:
# Affichage des premières lignes des données de notre base de données
data_clean.head()

,X,Y,OBJECTID,created_date,created_user,src_geo,clc_quartier,clc_secteur,id_arbre,haut_tot,...,nomfrancais,GlobalID,CreationDate,Creator,EditDate,Editor,feuillage,remarquable,lon,lat
0,1.720320e+06,8.294619e+06,1,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Richelieu,24.0,0.0,...,RAS,{476EB2CE-1FD4-4F89-8162-79D75651225A},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.282540,49.850458
1,1.720898e+06,8.293531e+06,2,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Léon Blum,24.0,0.0,...,RAS,{B5A9F630-92C5-4B8A-A934-0CABDA46085E},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.290521,49.840654
2,1.720894e+06,8.293542e+06,3,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Léon Blum,53.0,0.0,...,RAS,{F5914EAD-05CD-4ADF-A7C9-55EFF91B2ABE},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.290460,49.840756
3,1.720902e+06,8.293545e+06,4,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Léon Blum,54.0,0.0,...,RAS,{41168E06-B7C0-43CD-B8FE-7495B6E93AB5},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.290568,49.840783
4,1.721089e+06,8.293619e+06,5,2017-02-02,mickael.delaere,Orthophoto,Quartier du Centre-Ville,Boulevard Léon Blum,63.0,0.0,...,RAS,{4F0E4B12-4612-4F61-9911-43684831FD7D},2018-01-15,mickael.delaere,2018-01-15,mickael.delaere,NaN,Non,3.293178,49.841441


In [67]:
# Récupération des colonnes désirées 
data = data_clean[["haut_tot","haut_tronc","tronc_diam","fk_arb_etat","fk_stadedev","age_estim", "fk_prec_estim","clc_quartier", "clc_secteur","fk_port","fk_pied","fk_situation","fk_revetement","feuillage"]]

In [68]:
# Autre méthode pour récuperer les colonnes désirées
colonnes = ["haut_tot","haut_tronc","tronc_diam","fk_arb_etat","fk_stadedev","age_estim", "fk_prec_estim","clc_quartier", "clc_secteur","fk_port","fk_pied","fk_situation","fk_revetement","feuillage"]
data_new = data_clean[colonnes]

In [69]:
# Corrélation entre les variables numériques
correlation = data.corr(method='pearson', numeric_only= True).style.background_gradient(cmap='Pastel1')
correlation

,haut_tot,haut_tronc,tronc_diam,age_estim,fk_prec_estim
haut_tot,1.000000,0.441569,0.678304,0.678828,0.463950
haut_tronc,0.441569,1.000000,0.327135,0.505737,0.292037
tronc_diam,0.678304,0.327135,1.000000,0.857629,0.508601
age_estim,0.678828,0.505737,0.857629,1.000000,0.855497
fk_prec_estim,0.463950,0.292037,0.508601,0.855497,1.000000


In [70]:
# Binarisation des valeurs de fk_arb_etat

data.loc[data["fk_arb_etat"] == "Essouché","fk_arb_etat"] = 1
data.loc[data["fk_arb_etat"] != 1,"fk_arb_etat"] = 0
data.fk_arb_etat = data.fk_arb_etat.astype(int)

C:\Users\mathi\AppData\Local\Temp\ipykernel_16532\1889407845.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fk_arb_etat = data.fk_arb_etat.astype(int)


In [71]:
# Changement des ordinalités dans les colonnes en charactère
encodeur = OrdinalEncoder()
cols = ["clc_quartier", "clc_secteur","fk_port","fk_pied","fk_situation","fk_revetement","feuillage"]
changement = data[cols]
data[cols] = encodeur.fit_transform(changement)

C:\Users\mathi\AppData\Local\Temp\ipykernel_16532\4238524506.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[cols] = encodeur.fit_transform(changement)


In [72]:
# Visualisation des données transformées
print(data)

      haut_tot  haut_tronc  tronc_diam  fk_arb_etat fk_stadedev  age_estim  \
0          0.0         0.0         0.0            0  Inexistant  11.170052   
1          0.0         0.0         0.0            0  Inexistant  11.170052   
2          0.0         0.0         0.0            0  Inexistant  11.170052   
3          0.0         0.0         0.0            0  Inexistant  11.170052   
4          0.0         0.0         0.0            0  Inexistant  11.170052   
...        ...         ...         ...          ...         ...        ...   
8968       3.0         2.0        15.0            0       jeune   4.864046   
8969       3.0         2.0        15.0            0       jeune   4.864046   
8970       3.0         2.0        15.0            0       jeune   4.864046   
8971       3.0         2.0        15.0            0       jeune   4.864046   
8972       4.0         2.0        15.0            0       jeune   4.831146   

      fk_prec_estim  clc_quartier  clc_secteur  fk_port  fk_pie

In [73]:
# Nouvelle corrélation entre les variables numériques
new_correlation = data.corr(method='pearson', numeric_only= True).style.background_gradient(cmap='Pastel1')
new_correlation

,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,age_estim,fk_prec_estim,clc_quartier,clc_secteur,fk_port,fk_pied,fk_situation,fk_revetement,feuillage
haut_tot,1.000000,0.441569,0.678304,-0.028608,0.678828,0.463950,-0.136610,0.046705,0.293363,0.132394,0.129323,0.041705,-0.084808
haut_tronc,0.441569,1.000000,0.327135,-0.027451,0.505737,0.292037,-0.154619,0.131434,0.322126,-0.017162,0.007163,0.067256,0.039836
tronc_diam,0.678304,0.327135,1.000000,-0.013521,0.857629,0.508601,-0.107253,0.025183,0.314422,0.071302,-0.003548,0.176629,-0.078979
fk_arb_etat,-0.028608,-0.027451,-0.013521,1.000000,0.022844,0.064528,0.005303,-0.020738,-0.022574,-0.005246,0.033842,0.016573,0.000352
age_estim,0.678828,0.505737,0.857629,0.022844,1.000000,0.855497,-0.221926,0.013047,0.388264,0.002707,-0.019708,0.150270,-0.021434
fk_prec_estim,0.463950,0.292037,0.508601,0.064528,0.855497,1.000000,-0.255631,-0.043638,0.309225,-0.057233,-0.032796,0.083814,0.022325
clc_quartier,-0.136610,-0.154619,-0.107253,0.005303,-0.221926,-0.255631,1.000000,-0.062421,-0.062579,0.052388,0.091267,-0.059426,-0.076519
clc_secteur,0.046705,0.131434,0.025183,-0.020738,0.013047,-0.043638,-0.062421,1.000000,0.124953,0.142237,0.098194,-0.030567,-0.120593
fk_port,0.293363,0.322126,0.314422,-0.022574,0.388264,0.309225,-0.062579,0.124953,1.000000,0.010709,-0.004842,-0.077186,-0.145204
fk_pied,0.132394,-0.017162,0.071302,-0.005246,0.002707,-0.057233,0.052388,0.142237,0.010709,1.000000,0.060879,-0.103451,-0.055975


In [74]:
# Classification des données en X

x = data[["haut_tot", "haut_tronc","tronc_diam","clc_quartier"]]
print(x)
print(x.shape)

      haut_tot  haut_tronc  tronc_diam  clc_quartier
0          0.0         0.0         0.0           7.0
1          0.0         0.0         0.0           7.0
2          0.0         0.0         0.0           7.0
3          0.0         0.0         0.0           7.0
4          0.0         0.0         0.0           7.0
...        ...         ...         ...           ...
8968       3.0         2.0        15.0           6.0
8969       3.0         2.0        15.0           7.0
8970       3.0         2.0        15.0           6.0
8971       3.0         2.0        15.0           6.0
8972       4.0         2.0        15.0           6.0

[8973 rows x 4 columns]
(8973, 4)


In [75]:
# Classification des données en Y
y = data[["fk_arb_etat"]]
print(y)
print(y.shape)

      fk_arb_etat
0               0
1               0
2               0
3               0
4               0
...           ...
8968            0
8969            0
8970            0
8971            0
8972            0

[8973 rows x 1 columns]
(8973, 1)


In [76]:
print(np.unique(y))

[0 1]


In [77]:
x_train, y_train = x[:6281],y[:6281]
x_test, y_test = x[6281:],y[6281:]

### Apprentissage Supervisé pour la classification

#### Classification binaire

In [78]:
randomforest = RandomForestClassifier(n_estimators= 200, random_state= 42)
randomforest.fit(x_train,y_train)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(n_estimators=200, random_state=42)

In [82]:
gridsearch = GridSearchCV(estimator=randomforest, param_grid= {'n_estimators' : [i*25 for i in range(1,20)]},cv= 5, n_jobs=-1)
meilleur_model = gridsearch.fit(x_train, y_train)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [83]:
print(gridsearch.best_estimator_)

RandomForestClassifier(n_estimators=175, random_state=42)


### Métrique pour la classification

In [87]:
taux = cross_val_score(gridsearch, x_train, y_train, cv = 3, scoring='accuracy')
print(taux)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

[0.95797517 0.97325692 0.97563306]


In [88]:
print(np.mean(taux))

0.9689550514267097


#### Matrice de confusion

In [90]:
predictions = cross_val_predict(gridsearch, x_train, y_train, cv = 3)
print(predictions)

C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\mathi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y 

[0 0 0 ... 0 0 0]


In [91]:
matrice = confusion_matrix(y_train,predictions, normalize = "true")
print(matrice)

[[0.98878414 0.01121586]
 [0.97674419 0.02325581]]


### Préparation du Script